In [1]:
import os
import pandas as pd
import xarray as xr
from pathlib import Path

In [6]:
# Load the SPI3 dataset and rename season dimension
ds_spi3 = xr.open_dataset("../workspace/hazards/Drought/SPI3/ensemble/ensemble_return_period_SPI3.nc")

# Rename season dimension values
season_mapping = {
    'DJF': 'Winter',
    'JJA': 'Summer', 
    'MAM': 'Spring',
    'SON': 'Autumn'
}

# Apply the mapping
ds_spi3_renamed = ds_spi3.assign_coords(season=[season_mapping[s] for s in ds_spi3.coords['season'].values])


# Save back to file
out_file = "../workspace/hazards/Drought/SPI3/ensemble/ensemble_return_period.nc"
ds_spi3_renamed.to_netcdf(out_file)

print("Season dimension renamed successfully!")
print(f"New seasons: {ds_spi3_renamed.coords['season'].values}")


Season dimension renamed successfully!
New seasons: ['Winter' 'Summer' 'Spring' 'Autumn']


In [2]:
# Load and process the dataset
with xr.open_dataset("../workspace/hazards/Compound/HI/ensemble/ensemble_return_period_HI_final.nc") as ds:
    # Keep only days_hot_total and rename it to HI
    data = ds[['days_hot_total']].rename({'days_hot_total': 'HI_days_hot_total'})
    # Load into memory
    data_loaded = data.load()


out_file = "../workspace/hazards/Compound/HI/ensemble/ensemble_return_period.nc"
data_loaded.to_netcdf(out_file)

print("File successfully updated!")


File successfully updated!


In [3]:

# Define the base directory containing NC files
base_dir = Path("../workspace/hazards")

# Find all NC files recursively
nc_files = list(base_dir.rglob("ensemble_return_period.nc"))

print(f"Found {len(nc_files)} NC files to convert")

# Convert each NC file to CSV
for nc_file in nc_files:
    if \
        "SPI3" in str(nc_file) or \
        "HI" in str(nc_file) or \
        "Fire" in str(nc_file):
        try:
            print(f"Converting: {nc_file}")
            
            # Open the NetCDF file
            ds = xr.open_dataset(nc_file)
            
            # Convert to DataFrame and clean up
            df = ds.to_dataframe().dropna().reset_index()
            
            # Extract hazard indicator from folder path (e.g., "HI" from "hazards/Compound/HI/ensemble/...")
            path_parts = nc_file.parts
            hazard_indicator = None
            
            # Look for the indicator name in the path structure
            # Expected structure: .../hazards/HazardType/IndicatorName/ensemble/...
            if 'hazards' in path_parts:
                hazards_index = path_parts.index('hazards')
                if hazards_index + 2 < len(path_parts):
                    hazard_indicator = path_parts[hazards_index + 2]  # Skip 'hazards' and hazard type
                    print(f"  -> Extracted hazard indicator '{hazard_indicator}' from path")
            
            # Get the data variable name (should be the only data variable in the dataset)
            data_vars = [var for var in ds.data_vars]
            if len(data_vars) == 1:
                data_var_name = data_vars[0]
                
                # Use extracted hazard indicator or fall back to data variable name
                indicator_name = hazard_indicator if hazard_indicator else data_var_name
                
                # Create new columns: hazard_indicator and hazard_intensity
                df['hazard_indicator'] = indicator_name
                df['hazard_intensity'] = df[data_var_name]
                
                # Remove the original data variable column
                df = df.drop(columns=[data_var_name])
                
                print(f"  -> Data variable '{data_var_name}' restructured using indicator '{indicator_name}'")
            else:
                print(f"  -> Warning: Found {len(data_vars)} data variables, expected 1. Using original structure.")
            
            # Create output CSV file path - replace workspace with tests/tests_data, keep rest of structure
            # Get the relative path from the base_dir
            relative_path = nc_file.relative_to(base_dir)
            # Replace the first part of the path
            csv_relative_path = relative_path.with_suffix('.csv')
            # Create output in tests/tests_data with same structure
            csv_file = Path("../tests/tests_data/hazards") / csv_relative_path
            csv_file.parent.mkdir(parents=True, exist_ok=True)  # Create directories if needed
            
            # Save as CSV
            df.to_csv(csv_file, index=False)
            
            print(f"  -> Saved as: {csv_file}")
            
            # Close the dataset to free memory
            ds.close()
            
        except Exception as e:
            print(f"Error converting {nc_file}: {e}")

print("Conversion complete!")

Found 7 NC files to convert
Converting: ../workspace/hazards/Fire/FWI/ensemble_return_period.nc
  -> Extracted hazard indicator 'FWI' from path
  -> Data variable 'FWI_max' restructured using indicator 'FWI'
  -> Saved as: ../tests/tests_data/hazards/Fire/FWI/ensemble_return_period.csv
Converting: ../workspace/hazards/Fire/days_danger_total/ensemble_return_period.nc
  -> Extracted hazard indicator 'days_danger_total' from path
  -> Data variable 'days_danger_total' restructured using indicator 'days_danger_total'
  -> Saved as: ../tests/tests_data/hazards/Fire/days_danger_total/ensemble_return_period.csv
Converting: ../workspace/hazards/Compound/HI/ensemble/ensemble_return_period.nc
  -> Extracted hazard indicator 'HI' from path
  -> Data variable 'HI_days_hot_total' restructured using indicator 'HI'
  -> Saved as: ../tests/tests_data/hazards/Compound/HI/ensemble/ensemble_return_period.csv
Converting: ../workspace/hazards/Drought/SPI3/ensemble/ensemble_return_period.nc
  -> Extracted h